In [1]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor
from feature import Feature

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import json
from pathlib import Path
def read_conv(path:str, datalist:list):
    convs = []
    for p in datalist:
        datapath = Path(path + p + '/')
        for file_ in datapath.glob("*.json"):
            conv = []
            with open(file_, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    sp = t["speaker"]
                    utt = t["utterance"]
                    errors = t["error_category"]
                    type_ = t["type"]
                    one = Utterance(did, sp, utt, errors, type_)
                    conv.append(one)
            convs.append(conv)
    return convs      

In [3]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']


In [4]:
convs = read_conv(path, datalist)

In [5]:
# 前向き発話以外は後ろ向きと考える
def extract_response(convs, type_):
    type_utt = []
    forward = []
    backward = []
    plain = []
    for conv in convs:
        for i, ut in enumerate(conv):
            if i == len(conv)-1:
                continue
            # システムの質問に対する応答
            # if ut.is_system() and ut.is_type_included(type_) and not ut.is_exist_error():
            #     if not ut.is_utt_level_error():
            #         if conv[i+1].is_exist_type():
            #             print("後ろ+前", conv[i+1], conv[i+1].did)
            #         else:
            #             print("後ろ", conv[i+1])

            # システムの場合\
            
            if ut.is_type_included("挨拶"):
                continue
            utt = clean_text( ut.utt )
            utt = utt.replace("\u3000", " ")
            utt = utt.replace("？？", "？")
            if ut.is_system():
                # 発話レベルのエラーではなく
                if not ut.is_utt_level_error():
                    # 前向きである
                    if not ut.is_exist_error():
                        if ut.is_exist_type():
                        # # 後ろ+前ではない
                        # if not conv[i-1].is_exist_type():
                            forward.append(utt)
                        else:
                            plain.append(utt)
                            pass
                        
            # ユーザ発話
            else:
                # 前向き
                if ut.is_exist_type():
                    if conv[i-1].is_exist_type():
                        forward.append(utt)
                # ユーザで，後ろ向き
                else:
                    # 直前の発話は
                    if conv[i-1].is_exist_type():
                        backward.append(utt)
                    # else:
                    #     backward.append(utt)

            
            # システムの前向き 
            # if ut.is_system() and not ut.is_utt_level_error():
            #     if ut.is_exist_type():
            #         print("前向き", ut)
            #         # 後ろ+前
            #         if conv[i+1].is_exist_type():
            #             print("後ろ+前", conv[i+1], conv[i+1].did)
            #         # 後ろ
            #         else:
            #             print("後ろ", conv[i+1])
            #         print()
    
            #     else:
            #         if not ut.is_exist_error():
            #             print("not type", conv[i+1])
            #             print()
                    
    return forward, backward, plain

In [6]:
forward, backward, plain = extract_response(convs, "提案")

In [7]:
X = (forward+backward)
y = np.concatenate( [ np.zeros(len(forward))+1, np.zeros(len(backward))] )

In [8]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [9]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [10]:
F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)


300


In [11]:
F_path = "../X_y_data/response/"
F_name = "forback1.pickle"
featureM = DataManager(F_path)
featureM.save_data(F_name, F)

success save : ../X_y_data/response/forback1.pickle


In [12]:
print(F.feature_num)
X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

7134


In [13]:
# lr = LogisticRegression(solver='sag', max_iter=10000)
# lr.fit(X_train, y_train)

In [14]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [15]:
# y_pred = lr.predict(X_test)
y_pred = clf.predict(X_test)

In [16]:

for y_p, x_s in zip(y_pred[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

1.0 : あまりやりません。
0.0 : 私は上手ですよ。あなたはどうですか
0.0 : ボーっとするのが好きなので、嫌いではないですよ
1.0 : 年始は、旅行は行かず、帰省かな。
0.0 : こんにちは。音楽はどんなのを聴かれていますか？
1.0 : あまり酷くはありません。
0.0 : 映画は進みます？
1.0 : 朝の海も気持ちよさそうですね。
1.0 : 気をつけます。ありがとう。
1.0 : 今度は韓国ですね。
1.0 : 猫です
1.0 : 子供の相手で忙しいですよ
1.0 : ガンガン効いてるよ
1.0 : コブクロは全然聴かないんです。
0.0 : お好み焼き？！いいですねぇー。関西風ですか？!
0.0 : 材料とか作り方に違いがあるのかもしれません。
0.0 : 全くしないですね。どんな所が苦手ですか？
1.0 : テナーサックスはやっていませんよ
1.0 : 落ちていません
0.0 : 目眩は少ないですか
1.0 : 行きますよ。
1.0 : ニュージーランドかな。
0.0 : 今日も暑いですね、雨は降ってませんか?
1.0 : そうですよね〜
1.0 : 私は7時です。
1.0 : 毎朝ヨーグルトに入れて食べるようにしてます。バナナがすきですよ。
0.0 : スポーツは苦手なのでしてません。あなたは好きなスポーツがあるのかな？
0.0 : ランギングが好きです。あなたは？
0.0 : どんな本を読みますか？
0.0 : どちらも知らない人です。どんな職業の人なのですか？


In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[55  4]
 [ 9 76]]
accuracy =  0.9097222222222222
precision =  0.95
recall =  0.8941176470588236
f1 score =  0.9212121212121211


In [25]:
model_path = "../models/response/"
model_name = "forback_clf.pickle"
modelM = DataManager(model_path)
print(model_name)
modelM.save_data(model_name, clf)

forback_clf.pickle
success save : ../models/response/forback_clf.pickle


In [19]:
# 本当に質問を識別できるか
from tqdm import tqdm
y_ = []
y_pred_ = [] 
for conv in tqdm(convs):
    for i, ut in enumerate(conv):
        f = F.featurization(ut.utt)
        y_pred_.append( clf.predict(f.reshape(1, -1)) )
        if ut.is_exist_type() and not ut.is_type_included("挨拶"):
            y_.append(1)
        else:
            y_.append(0)

100%|██████████| 200/200 [08:33<00:00,  2.57s/it]


In [24]:
print('confusion matrix = \n', confusion_matrix(y_true=y_, y_pred=y_pred_))
print('accuracy = ', accuracy_score(y_true=y_, y_pred=y_pred_))
print('precision = ', precision_score(y_true=y_, y_pred=y_pred_))
print('recall = ', recall_score(y_true=y_, y_pred=y_pred_))
print('f1 score = ', f1_score(y_true=y_, y_pred=y_pred_))

confusion matrix = 
 [[2821  443]
 [ 147  789]]
accuracy =  0.8595238095238096
precision =  0.640422077922078
recall =  0.842948717948718
f1 score =  0.727859778597786


In [25]:
y_[:10]

[0, 0, 1, 0, 0, 1, 0, 1, 0, 1]

In [26]:
y_pred_[:10]

[0, 0, 0, 0, 0, 1, 0, 1, 0, 1]

In [22]:
y_pred_ = [1 if p==0 else 0 for p in y_pred_]

In [23]:
X_ = []
for conv in convs:
    for i, ut in enumerate(conv):
        X_.append(ut.utt)

In [52]:
correct_n = 0
bad_n = 0
all_tp_one = 0
for x, t, n in zip(X_, y_, y_pred_):
    # t==1 : 本当の質問
    # n==1 : 予想された質問
    if t == 1:
        all_tp_one += 1
        if n == 1:
            # t==n==1 : 適切に検出
            correct_n += 1
            # print(x)
        else:
            # 破綻なのに未検出
            # 質問を未検出
            bad_n += 1
            # print(x)
    else:
        # 破綻ではないのに破綻扱い
        # 間違い
        if n == 1:
            bad_n += 1
            print(x)
print("corrent_n:", correct_n)
print("all_tp_one:", all_tp_one)
print("bad_n:", bad_n)
print("rate c:", correct_n/all_tp_one)

そうですか
焼きそばは好きですか。麺が美味しいですね
魚はおいんですか？？海は素晴らしいですね
ボーっとするのが好きなので、嫌いではないですよ
大分別府はいんですか？？大分別府まで九州横断特急に乗りますよねー
しりとりは一緒に盛り上がりましょうか？
恋の季節って、やっぱり夏なのかなあ……。
今は、してないですね。
今は、してないですね。
納豆は最高ですか。食べるのがいいですね
スイカは大好きですか。スイカは実が大きいですね
メロンは好きですか。スイカは大好きですね
え〜欲しいの？わたしは要らないな
元気ですかは元気です
元気は良いんですか？？元気ですかは元気か
その気持ち凄く分かる。
思い出したんですが、読書する習慣があるって言ってましたよね？歴史小説を読みたいですよねー
そうなの？すごいね。
甘いんですか？？果物はカロリーが高いですよねー
少子化問題が気になります。
ありがとう。あなたもお気をつけて。
熱中症に気をつけか？？
熱中症で死者が出てるんでしょうか
オリンピックは映画に対抗する気はないと思いますよ。
そうなんですか。知りませんでした
そうなんですか
そうだね。分かる分かる！
素通りを許されてないな
誰かの役に立っていると思えば、興味のない仕事でもおもしろくなるものですよ。
海は気持ちよさそうですか。沖縄で行きます？
沖縄の人は泳ぎが苦手って聞いたけど、本当ならおもしろいよね。
最新の洋楽って、最近の人はあまり聞かないのかなあ。
今は、してないですね。
お弁当ならコンビニに売ってますよ。
こんばんはー。熱中症に気をつけて。
健康が一番だよね。君は結婚してるのかい？
こんにちは。水分を補給してるから、熱中症にはならないよ。
エコは大事ですか。エコは大切ですね
まだ今年は海へ行ってませんね。
いいんですか
砂浜は熱いんですか？？やけどが深いですよねえ
天気はいんですか？？全身に負いますよねー
透明度と言えば、沖縄の海の透明度はびっくりするほどでした
血液型占いって、本当に当たるのかな。
そんなこと言わないでください。話題を変えましょう、今、行ってみたい場所はどこですか？
日本人の８割が人生の間で腰痛に悩むらしいよ。
意外と歴史は古くて、１９４１年ごろ、福岡の中洲で作られたんだ。
冷凍は美味しいんですか？果物が良いですね
ほんとうですか。
海はいんですか？？海は

In [53]:
from feature import Feature

F_path = "../X_y_data/response/"
F_name = "typeClassify_F2.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

success load : ../X_y_data/response/typeClassify_F2.pickle


In [54]:
F.set_preprocessor(Preprocessor())

300


In [55]:
model2_path = "../models/response/"
model2_name = "typeClassify_SVC.pickle"
model2M = DataManager(model2_path)
lr = modelM.load_data(model_name)

success load : ../models/response/forback_clf.pickle
